In [1]:
pip install git+https://github.com/pathpy/pathpyG.git

  Cloning https://github.com/pathpy/pathpyG.git to /private/var/folders/bq/5f7dchhj4612jj76hb16wtq00000gn/T/pip-req-build-06dy_6k2
  Running command git clone --filter=blob:none --quiet https://github.com/pathpy/pathpyG.git /private/var/folders/bq/5f7dchhj4612jj76hb16wtq00000gn/T/pip-req-build-06dy_6k2
  Resolved https://github.com/pathpy/pathpyG.git to commit db292cb2fbedd0b119d5046df5c25d807cc91894
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 11.6 MB/s eta 0:00:00
  Created wheel for pathpyG: filename=pathpyg-0.2.0-py3-none-any.whl size=201222 sha256=d92930acf568a980ac5f5f6a91e089e320d93f0b5385fffe484a76f09231af6b
  Stored in directory: /private/var/folders/bq/5f7dchhj4612jj76hb16wtq00000gn/T/pip-ephem-wheel-cache-8_4g490o/wheels/98/3f/d9/b5d6b74f0678695e27ff5a26cdf81736966c3ec8fb99b39c26
Successfully built pathpyG

[notice] A new releas

In [2]:
pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pathpyG as pp
import pandas as pd

In [8]:
df = pd.read_csv('out.mit', sep='\t', header=None, skiprows=1, engine='python')

df.drop(columns=1, inplace=True)


print(df.head())

     0           2
0  1 2  1095183096
1  3 4  1095183096
2  5 6  1095183096
3  7 8  1095183096
4  9 2  1095183096


In [9]:
df.head()

,0,2
0,1 2,1095183096
1,3 4,1095183096
2,5 6,1095183096
3,7 8,1095183096
4,9 2,1095183096


In [18]:
import pandas as pd
from collections import defaultdict, deque

# Assuming df is already loaded and contains two columns: source and destination

# Add a placeholder timestamp (e.g., using the same value for all rows)
df['timestamp'] = 1095183096  # Replace with actual logic if available, or use a constant timestamp

# Rename the columns to make them more readable
df.columns = ['source', 'destination', 'timestamp']

# Clean the data: Drop rows with missing values
df_cleaned = df[['source', 'destination', 'timestamp']]  # Select relevant columns
df_cleaned.dropna(inplace=True)  # Drop rows with missing values

# Convert cleaned DataFrame to list of tuples (source, destination, timestamp)
data_tuples = list(df_cleaned.itertuples(index=False, name=None))

# Define the PaCo algorithm
def paco(data, delta, max_length):
    """
    Implements the PaCo algorithm for counting causal paths in temporal networks.
    
    Args:
    - data: List of tuples (source, destination, timestamp).
    - delta: Maximum time difference between consecutive links.
    - max_length: Maximum path length (K).
    
    Returns:
    - path_counts: Dictionary of causal paths and their counts.
    """
    # Sort data by timestamp
    data = sorted(data, key=lambda x: x[2])
    
    # Initialize sliding window and path counters
    window = deque()  # Sliding window
    path_counts = defaultdict(int)  # Overall causal path counts
    
    # Iterate through each link in the data
    for (s, d, t) in data:
        current_counts = defaultdict(int)  # Local counts for current link
        current_counts[(s, d)] = 1  # A single link is a causal path of length 1
        
        # Update the sliding window: remove outdated links
        while window and window[0][2] < t - delta:
            window.popleft()
        
        # Extend paths from the sliding window
        for (prev_s, prev_d, prev_t, prev_counts) in window:
            if prev_d == s and prev_t < t:  # Valid path extension
                for path, count in prev_counts.items():
                    if len(path) < max_length:  # Enforce maximum path length
                        extended_path = path + (d,)
                        current_counts[extended_path] += count
        
        # Update global counts
        for path, count in current_counts.items():
            path_counts[path] += count
        
        # Add current link and its paths to the sliding window
        window.append((s, d, t, current_counts))
    
    return path_counts

# Parameters
delta = 30  # Example: maximum time difference
max_length = 4  # Maximum path length

# Run the algorithm with the cleaned data
results = paco(data_tuples, delta, max_length)

# Output the results
for path, count in results.items():
    print(f"Path: {path}, Count: {count}")


KeyboardInterrupt: 